In [66]:
from bs4 import BeautifulSoup
import requests
import csv
import re

mainurl = "http://swbiodiversity.org/seinet"
imgurl = "http://swbiodiversity.org"
url = "http://swbiodiversity.org/seinet/collections/listtabledisplay.php"
indiviual = 'http://swbiodiversity.org/seinet/collections/individual/index.php?occid={}&clid=0'
data = "usethes=1&taxontype=2&taxa=Cactaceae&country=&state=&county=&local=&elevlow=&elevhigh=&upperlat=&upperlat_NS=N&bottomlat=&bottomlat_NS=N&leftlong=&leftlong_EW=W&rightlong=&rightlong_EW=W&footprintwkt=&pointlat=&pointlat_NS=N&pointlong=&pointlong_EW=W&radius=&radiusunits=km&collector=&collnum=&eventdate1=&eventdate2=&catnum=&includeothercatnum=1&reset=1&db=all"
html_content = requests.get(url, data).text


soup = BeautifulSoup(html_content, "lxml")
row = soup.findAll("tr",  attrs={"class": "alt"})
output_rows = []
for a in row:
    try:
        cells = a.findAll("td")
        #print (cells)
        id_num = cells[0].a.text
        scientific_name = cells[4].find(text=True)
        elevation = cells[14].find(text=True)
        habitat = cells[12].find(text=True)
        #(cells[11].find(text=True).strip().rsplit(', ',1)[1] if cells[11].find(text=True) is not None else " ")
        if num_there(cells[11].find(text=True).strip()) :
            latitude = cells[11].find(text=True).strip().rsplit(', ')[-1].split(' ')[0]
            longitude = cells[11].find(text=True).strip().rsplit(', ')[-1].split(' ')[-1]
        
        country =  cells[8].find(text=True)
        state =  cells[9].find(text=True)
        county =  cells[10].find(text=True)
        html_content_new = requests.get(indiviual.format(cells[0].a.text)).text
        #print (html_content_new)
        soup_new = BeautifulSoup(html_content_new, "lxml")
        
        image = soup_new.find('b',text= 'Specimen Images').parent.parent
        #print(image.div.a.img['src'])
        #print (soup_new)
        catalogue = soup_new.find('b',text= 'Catalog #:').next_element.next_element.strip() if soup_new.find('b',text='Catalog #:') is not None else ''
        taxon = soup_new.find('b',text='Taxon:').next_element.parent.parent.i.text if soup_new.find('b',text='Taxon:') is not None else ''
        Family = soup_new.find('b',text='Family:').next_element.next_element.strip() if soup_new.find('b',text='Family:') is not None else ''
        genus = soup_new.find('b',text='Genus:').next_element.next_element.strip() if soup_new.find('b',text='Genus:') is not None else ''
        date = soup_new.find('b',text='Date: ').next_element.next_element.strip() if soup_new.find('b',text='Date: ') is not None else ''
        locality = soup_new.find('b',text='Locality:').next_element.next_element.strip() if soup_new.find('b',text='Locality:') is not None else ''
        
        output_row = []
        if image is None:
            pass
        else :
            urls = image.div.a.img['src']
            if 'http' in urls:
                output_row.append(image.div.a.img['src'])
            else:
                output_row.append(imgurl+image.div.a.img['src'])
            output_row.append(id_num)
            output_row.append(scientific_name)
            output_row.append(catalogue)
            output_row.append(taxon)
            output_row.append(Family)
            output_row.append(date)
            output_row.append(country)
            output_row.append(state)
            output_row.append(county)
            output_row.append(locality)
            output_row.append(elevation)
            output_row.append(latitude)
            output_row.append(longitude)
            output_row.append(habitat)
            
            print(output_row)
            output_rows.append(output_row)
    except NavigableString: 
            pass
    


with open('output2.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)

['http://swbiodiversity.org/imglib/seinet/mam/PH/PH00777/PH00777294_1581450851_tn.jpg', '24536701', 'Opuntia basilaris', 'PH00777294', 'Opuntia basilaris', 'Cactaceae', '1938-05-15', 'United States, Nevada, Clark, Kyle Canyon Fan', '1300', '36.294687', '-115.445289', 'desert slope, larrea belt']
['http://swbiodiversity.org/imglib/seinet/mam/PH/PH00777/PH00777296_1581452205_tn.jpg', '24536703', 'Opuntia basilaris', 'PH00777296', 'Opuntia basilaris', 'Cactaceae', '1881-03-00', 'United States, California, Riverside, Whitewater', None, '33.925232', '-116.63866', 'desert-hills']
['http://swbiodiversity.org/imglib/seinet/mam/PH/PH00777/PH00777307_1581451187_tn.jpg', '24536714', 'Opuntia chlorotica', 'PH00777307', 'Opuntia chlorotica', 'Cactaceae', '1920-01-06', 'United States, Arizona, Pima, Sabino Canyon, Santa Catalina Mountains', '914', '32.322367', '-110.809822', None]
['http://swbiodiversity.org/imglib/seinet/mam/PH/PH00777/PH00777370_1581450784_tn.jpg', '24536777', 'Opuntia fragilis', 

NameError: name 'NavigableString' is not defined

In [53]:
def num_there(s):
    if s:  
        try : 
            return s[-1].isdigit()
        except NavigableString:
            pass
    else :
        pass